In [7]:
import pandas as pd
import re

In [23]:
df_data = pd.read_csv("sample_text.csv")

In [24]:
df_data

,note_id,hadm_id,discharge_instructions,brief_hospital_course,bhc_preceding_text,age,sex,cc,diags,procs,prescriptions,labs,microbio
0,1234,22841357,SAMPLE INJECTION,SAMPLE INJECTION,SAMPLE INJECTION,SAMPLE INJECTION,SAMPLE INJECTION,SAMPLE INJECTION,SAMPLE INJECTION,SAMPLE INJECTION,SAMPLE INJECTION,SAMPLE INJECTION,SAMPLE INJECTION


In [4]:
df_example = pd.read_csv("similarity_scores_100.csv")

In [5]:
df_example

,Unnamed: 0,hadm_id,preceding_text,note_id,discharge_instructions,brief_hospital_course,discharge_instructions_word_count,brief_hospital_course_word_count,cosine_similarity_score,spacy_similarity_score,rouge1
0,0,22841357,\nName: ___ Unit No: _...,10000032-DS-22,"Dear ___,\n\n___ was a pleasure to take care o...","___ with HIV on HAART, HCV cirrhosis with asci...",115,503,0.539475,0.852054,{'rouge1': Score(precision=0.49079754601226994...
1,1,22927623,\nName: ___ Unit No: ___\n...,10000117-DS-21,"Dear Ms. ___,\n\nYou were hospitalized at ___....",Ms. ___ is a ___ with history of GERD who pres...,59,73,0.420363,0.509862,"{'rouge1': Score(precision=0.7681159420289855,..."
2,2,21738619,\nName: ___ Unit No: ___...,10000935-DS-19,"Dear ___, \n\n___ were seen in the ED for ongo...",___ Female with PMH significant for depression...,105,322,0.238139,0.688652,"{'rouge1': Score(precision=0.3375, recall=0.15..."
3,3,25849114,\nName: ___ Unit No: ___...,10000935-DS-21,"Dear Ms. ___,\nIt was a pleasure taking part i...",Ms. ___ is a ___ with metastatic cancer of unk...,173,57,0.422005,0.866834,"{'rouge1': Score(precision=0.8363636363636363,..."
4,4,29654838,\nName: ___ Unit No: ___\n \nAdmi...,10000980-DS-20,"Dear Ms. ___, \n\nYou were admitted for shortn...","___ woman with h/o hypertension, hypelipidemia...",174,459,0.326648,0.835297,"{'rouge1': Score(precision=0.3839479392624729,..."
...,...,...,...,...,...,...,...,...,...,...,...
95,95,22539296,\nName: ___ Unit No: _...,10012853-DS-9,"Dear Ms. ___, \n\nIt was a pleasure caring for...",Mrs. ___ is a ___ y/o woman with a PMH of atri...,131,388,0.454588,0.902810,{'rouge1': Score(precision=0.49868073878627966...
96,96,24173031,\nName: ___ Unit No: ___...,10013015-DS-4,"Dear Ms. ___,\n\nThank you for allowing us to ...",Ms. ___ is an ___ female with history of A. fi...,269,913,0.607951,0.907609,"{'rouge1': Score(precision=0.5542299349240781,..."
97,97,22098926,\nName: ___ Unit No: ___\n \nAdmission...,10013310-DS-17,"Dear Ms. ___ and family, \n\nIt was a pleasur...",BRIEF SUMMARY:\n==============\nMs. ___ is a _...,201,204,0.481185,0.905386,"{'rouge1': Score(precision=0.4563106796116505,..."
98,98,23404838,\nName: ___ Unit No: ___\n \nAd...,10013502-DS-19,"Dear Mr. ___,\n\nIt was a pleasure taking care...",___ with DMII c/b neuropathy here with a foot ...,105,304,0.514895,0.951858,{'rouge1': Score(precision=0.49310344827586206...


In [9]:
def check_subheading_info(document, subheading):
    # Split the document into lines
    lines = document.split("\n")
    
    # Iterate through each line and the next line to check for missing info
    for i in range(len(lines) - 1):
        current_line = lines[i].strip()
        next_line = lines[i + 1].strip()
        
        # Check if the current line is the subheading
        if current_line == subheading:
            # Check if the next line is not empty or does not contain only placeholder
            if next_line and not re.match(r'^_+$', next_line):
                return 0  # information is present and valid
            
            return 1
    
    # Return 1 if subheading is not found at all
    return 1

In [38]:
def inject_subheading_info(document, subheadings, dataframe_info):
    # Split the document into lines
    lines = document.split("\n")
    
    # Initialize variables to build the new document
    found_subheading = False
    
    # Iterate through each line to find the subheading
    for i in range(len(lines) - 1):
        current_line = lines[i].strip()
        
        # Iterate through all the possible variants in the subheadings list
        for subheading in subheadings:
            # Check if the current line is the subheading
            if current_line == subheading:
                found_subheading = True
                return document

    # Check if subheading was never found, add it at the end if needed
    if not found_subheading:
        document = document + "\n" + subheadings[0]
        document = document + "\n" + dataframe_info

    # Join all lines to form the new document
    return document

In [42]:
def data_injection(hadm_id, document, data):
    '''
    Currently Implemented Subheadings:
    age	sex	cc	diags	procs	prescriptions	labs	microbio
    1. cc - Chief Complaint:
    2. procs - Major Surgical or Invasive Procedure:
    3. labs - Pertinent Results:/LABS ON ADMISSION:/ADMISSION LABS/LABS ON DISCHARGE:
    4. microbio - MICRO:/URINE CULTURE/GRAM STAIN/FLUID CULTURE/ANAEROBIC CULTURE/NO MICROORGANISMS SEEN./Microbiology:
    5. diags - this is not well structured, usually pretext and the preceding text always includes a one-liner in first part of HPI
    6. radiology - Imaging:/"CT ____(specific)"/CXR
    7. prescriptions seem to never be mentioned in preceding text?
    '''
    
    #Making sure there is injection data available for specific hadm_id
    if not (df_data['hadm_id'] == hadm_id).empty:
        
        #grabbing the information from the specific hadm_id
        row = data.loc[data['hadm_id'] == hadm_id]
        
        for i in df_example.index:
            cc = ["Chief Complaint:"]
            procs = ["Major Surgical or Invasive Procedure:"]
            labs = ["Pertinent Results:", "LABS ON ADMISSION:", "ADMISSION LABS", "LABS ON DISCHARGE:"]
            microbio = ["MICRO:", "URINE CULTURE", "GRAM STAIN", "FLUID CULTURE", "ANAEROBIC CULTURE", "NO MICROORGANISMS SEEN.", "Microbiology:"]

            #Injecting Chief Complaint
            document = inject_subheading_info(document, cc, row.loc[0]["cc"])
            document = inject_subheading_info(document, procs, row.loc[0]["procs"])
            document = inject_subheading_info(document, labs, row.loc[0]["labs"])
            document = inject_subheading_info(document, microbio, row.loc[0]["microbio"])
    else:
        print("No data available for injection.")
    return document

In [43]:
test_1 = data_injection(22841357, df_example.loc[0]["preceding_text"], df_data)

In [44]:
test_1

" \nName:  ___                     Unit No:   ___\n \nAdmission Date:  ___              Discharge Date:   ___\n \nDate of Birth:  ___             Sex:   F\n \nService: MEDICINE\n \nAllergies: \nPercocet\n \nAttending: ___.\n \nChief Complaint:\nabdominal fullness and discomfort\n \nMajor Surgical or Invasive Procedure:\n___ diagnostic paracentesis\n___ therapeutic paracentesis\n\n \nHistory of Present Illness:\n___ with HIV on HAART, COPD, HCV cirrhosis complicated by \nascites and HE admitted with abdominal distention and pain. She \nwas admitted to ___ for the same symptoms \nrecently and had 3L fluid removed (no SBP) three days ago and \nfelt better. Since discharge, her abdomen has become \nincreasingly distended with pain. This feels similar to prior \nepisodes of ascites.  \nHer diuretics were recently decreased on ___ due to worsening \nhyponatremia 128 and hyperkalemia 5.1. Patient states she has \nbeen compliant with her HIV and diuretic medications but never \nfilled out the 

In [45]:
df_example.loc[0]["preceding_text"]

" \nName:  ___                     Unit No:   ___\n \nAdmission Date:  ___              Discharge Date:   ___\n \nDate of Birth:  ___             Sex:   F\n \nService: MEDICINE\n \nAllergies: \nPercocet\n \nAttending: ___.\n \nChief Complaint:\nabdominal fullness and discomfort\n \nMajor Surgical or Invasive Procedure:\n___ diagnostic paracentesis\n___ therapeutic paracentesis\n\n \nHistory of Present Illness:\n___ with HIV on HAART, COPD, HCV cirrhosis complicated by \nascites and HE admitted with abdominal distention and pain. She \nwas admitted to ___ for the same symptoms \nrecently and had 3L fluid removed (no SBP) three days ago and \nfelt better. Since discharge, her abdomen has become \nincreasingly distended with pain. This feels similar to prior \nepisodes of ascites.  \nHer diuretics were recently decreased on ___ due to worsening \nhyponatremia 128 and hyperkalemia 5.1. Patient states she has \nbeen compliant with her HIV and diuretic medications but never \nfilled out the 